# Run BQML queries

In [ ]:
!pip install --upgrade google-cloud-bigquery

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `consumer_risk.classification_model`
OPTIONS
(
model_type='logistic_reg',
labels = ['target_binary']
)
AS

SELECT
  CASE WHEN TARGET >0 THEN 'bad'
  ELSE 'good' END AS target_binary
  , credit_score
  , cltv
  , debt_to_income
  , ltv
FROM
  `freddie_mac_public_dataset.train_input_10M`
;

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `consumer_risk.classification_model`)

In [ ]:
%%bigquery
#standardSQL
SELECT * FROM 
  ML.EVALUATE(MODEL `consumer_risk.classification_model`, (
SELECT
  CASE WHEN TARGET >0 THEN 'bad'
  ELSE 'good' END AS target_binary
  , credit_score
  , cltv
  , debt_to_income
  , ltv
FROM
  `freddie_mac_public_dataset.train_input_10M`
  ))

In [ ]:
%%bigquery
#standardSQL
SELECT * FROM 
  ML.PREDICT(MODEL `consumer_risk.classification_model`, (
SELECT
   TARGET
  , credit_score
  , cltv
  , debt_to_income
  , ltv
FROM
  `freddie_mac_public_dataset.train_input_sample10k`
  ))
LIMIT 10

In [ ]:
%%bigquery
#standardSQL
SELECT 
  predicted_target_binary
  , AVG(credit_score) as average_credit_score
  , AVG(ltv) as average_ltv
FROM 
  ML.PREDICT(MODEL `consumer_risk.classification_model`, (
SELECT
   TARGET
  , credit_score
  , cltv
  , debt_to_income
  , ltv
FROM
  `freddie_mac_public_dataset.train_input_100M`
  ))
GROUP BY  predicted_target_binary
LIMIT 10000